In [60]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import random
import requests
import re
import os
import urllib.request

# List of user agents to rotate
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    # Add more user agents as needed
]


In [102]:
# Function to get HTML content from a URL
def get_html_content(url):
    headers = {'User-Agent': random.choice(USER_AGENTS)}  # Randomly select user agent

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to fetch URL: {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [110]:
# Function to extract timestamp and content from HTML
def extract_course_content(url):
    name = re.search(r'courses/(.*?)/', url).group(1)
    
    html_content = get_html_content(url)
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        videos = soup.find_all('a', class_='resource-thumbnail')
        lessons = soup.find_all('a', class_='resource-list-title')
        
        videos_txt = f"./{name}/{'videos_'}{name}.txt"
        titles_txt = f"./{name}/{'titles_'}{name}.txt"
        lessons_txt = f"./{name}/{'lesson_'}{name}.txt"
        
        with open(videos_txt, 'w') as txt_file:
            for i in range(0, len(videos)):
                txt_file.write(f"{videos[i]['href']}\n")
                
        with open(lessons_txt, 'w') as txt_file:
            for i in range(0, len(lessons)):
                txt_file.write(f"{'https://ocw.mit.edu/'}{lessons[i]['href']}\n")
                
        with open(titles_txt, 'w') as txt_file:
            for i in range(0, len(lessons)):
                txt_file.write(f"{lessons[i].text}\n")
                
        print(f"Data extracted and written successfully")
    else:
        print("Failed to extract HTML content.")
        
    return (f"./{name}/{'videos_'}{name}.txt", f"./{name}/{'titles_'}{name}.txt")

In [175]:
def get_video_transcript(url, location):
    chrome_options = Options()
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')
    driver = webdriver.Chrome()
    driver.get(url)
    
    driver.find_element(By.CLASS_NAME, value="tab-title-section").click()

    # Wait for some time for the page to load or elements to appear
    time.sleep(5)  # Wait for 5 seconds (adjust as needed)

    # Get HTML content after operations have been performed
    html_content = driver.page_source

    # # Extract timestamp and content from HTML
    # extract_timestamp_content(html_content, url)

    # Close the WebDriver
    driver.quit()
    
    # Use regular expression to extract the desired part
    name = url.split("/")[-2]
    
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        transcript_lines = soup.find_all('div', class_='transcript-line')
        filename = f"./{location}/{'transcripts'}/{name}.txt"
        with open(filename, 'w') as txt_file:
            for line in transcript_lines:
                timestamp = line.find('span', class_='transcript-timestamp').text.strip()
                content = line.find('span', class_='transcript-text').text.strip()
                content = " ".join(content.split("\n"))  # Remove newline characters
                txt_file.write(f"{timestamp} | {content}\n")
        print(f"Data extracted and written to {filename}")
    else:
        print("Failed to extract HTML content.")

In [112]:
def agent(course_url):
    course_name = re.search(r'courses/(.*?)/', course_url).group(1)
    
        # Check if the directory already exists
    if not os.path.exists(course_name):
        # Create the directory
        os.makedirs(course_name)
        os.makedirs(f"{course_name}/{'transcripts'}")
        os.makedirs(f"{course_name}/{'videos'}")
        print("Directory created successfully!")
    else:
        print("Directory already exists!")
        
    raw_data = extract_course_content(course_url)
    
    # videos = []
    # contents = []
    
    # with open(raw_data[0], 'r') as file:
    #     for line in file:
    #         videos.append(line.strip())
    
    # with open(raw_data[1], 'r') as file:
    #     for line in file:
    #         contents.append(line.strip())
            
    # for i in range(0, len(videos)):
    #     get_video_transcript(contents[i], course_name) 
    #     # name = contents[i].split("/")[-2]
    #     # urllib.request.urlretrieve(videos[i], f"{course_name}/{'videos'}/{name}.mp4") 

course_url = "https://ocw.mit.edu/courses/6-00-introduction-to-computer-science-and-programming-fall-2008/resources/lecture-videos/"
agent(course_url)
    

Directory already exists!
Data extracted and written successfully


In [111]:
def convert_to_srt(transcript):
    lines = transcript.split("\n")
    srt_lines = []
    index = 1
    for i in range(len(lines) - 1):
        line = lines[i].strip()
        next_line = lines[i + 1].strip()
        if line:
            start_time, content = line.split(" | ", 1)  # Limit split to 1 occurrence to handle content with "|"
            duration = next_line.split(" | ")[0]
            start_time_formatted = start_time.replace(".", ",")
            duration_formatted = duration.replace(".", ",")
            srt_lines.append(f"{index}\n{start_time_formatted} --> {duration_formatted}\n{content}\n")
            index += 1
    # Last line handling
    if lines:
        last_line = lines[-1].strip()
        if "|" in last_line:
            start_time, content = last_line.split(" | ", 1)
            start_time_formatted = start_time.replace(".", ",")
            srt_lines.append(f"{index}\n{start_time_formatted} --> 99:99:99,999\n{content}\n")
    return "".join(srt_lines)

# Given transcript
transcript = """
0:00 | The following content is provided under a Creative
0:02 | Commons license.
0:03 | Your support will help MIT OpenCourseware continue to
0:06 | offer high-quality educational resources for free.
0:10 | To make a donation, or view additional materials from
0:13 | hundreds of MIT courses, visit MIT OpenCourseware, at
0:17 | ocw.mit.edu .
"""

# Convert to SRT format
srt_content = convert_to_srt(transcript)
print(srt_content)

1
0:00 --> 0:02
The following content is provided under a Creative
2
0:02 --> 0:03
Commons license.
3
0:03 --> 0:06
Your support will help MIT OpenCourseware continue to
4
0:06 --> 0:10
offer high-quality educational resources for free.
5
0:10 --> 0:13
To make a donation, or view additional materials from
6
0:13 --> 0:17
hundreds of MIT courses, visit MIT OpenCourseware, at
7
0:17 --> 
ocw.mit.edu .



In [95]:
from datetime import datetime, timedelta

def convert_time(time_str):
    # Parse the time string
    minutes, seconds = map(int, time_str.split(":"))
    # Calculate total seconds
    total_seconds = minutes * 60 + seconds
    # Convert total seconds to timedelta object
    time_delta = timedelta(seconds=total_seconds)
    # Format the timedelta as hours:minutes:seconds,milliseconds
    formatted_time = (datetime.min + time_delta).strftime('%H:%M:%S,%f')[:-3]
    return formatted_time

# Example time string
time_str = "1:34"

# Convert time string
formatted_time = convert_time(time_str)
print(formatted_time)


00:01:34,000


In [98]:
def convert_to_srt(transcript):
    lines = transcript.strip().split("\n")
    srt_lines = []
    index = 1
    start_time = ""
    prev_line_full_stop = False
    i = 0

    while i in range(0, len(lines)):
        line = lines[i]
        timestamp, content = line.split(" | ", 1)
        
        if i == 0 or prev_line_full_stop:
            start_time = timestamp
            prev_line_full_stop = False
        
        try:
            srt_lines.append((index, start_time, content, lines[i+1].split(" | ")[0]))
        except:
            srt_lines.append((index, start_time, content, "12:59"))
        
        
        if "." in content:
            prev_line_full_stop = True
            i+=1
            continue

        for j in range(i+1, len(lines)):
            next_content = lines[j].split(" | ")[1]
            if "." in next_content:
                try:
                    end_time = lines[j+1].split(" | ")[0]
                except:
                    end_time = "12:59"
                prev_line_full_stop = True
                srt_lines[-1] = (index, start_time, content + " " + next_content, end_time)
                index += 1
                i = j + 1
                break
            else:
                content = content + " " + next_content
    print(srt_lines)
    srt_content = ""
    for srt_line in srt_lines:
        index, start_time, content = srt_line[:3]
        end_time = srt_line[3]
        start_time = convert_time(start_time)
        end_time = convert_time(end_time)
        srt_content += f"{index}\n{start_time.replace('.', ',')} --> {end_time.replace('.', ',')}\n{content}\n\n"

    return srt_content

# Given transcript
transcript = """
0:00 | The following content is provided under a Creative
0:02 | Commons license.
0:03 | Your support will help MIT OpenCourseware continue to
0:06 | offer high-quality educational resources for free.
0:10 | To make a donation, or view additional materials from
0:13 | hundreds of MIT courses, visit MIT OpenCourseware, at
0:17 | ocw.mit.edu .
0:17 | PROFESSOR: Good morning.
"""

# Convert to SRT format
srt_content = convert_to_srt(transcript)
print(srt_content)


[(1, '0:00', 'The following content is provided under a Creative Commons license.', '0:03'), (2, '0:03', 'Your support will help MIT OpenCourseware continue to offer high-quality educational resources for free.', '0:10'), (3, '0:10', 'To make a donation, or view additional materials from hundreds of MIT courses, visit MIT OpenCourseware, at ocw.mit.edu .', '0:17'), (4, '0:17', 'PROFESSOR: Good morning.', '12:59')]
1
00:00:00,000 --> 00:00:03,000
The following content is provided under a Creative Commons license.

2
00:00:03,000 --> 00:00:10,000
Your support will help MIT OpenCourseware continue to offer high-quality educational resources for free.

3
00:00:10,000 --> 00:00:17,000
To make a donation, or view additional materials from hundreds of MIT courses, visit MIT OpenCourseware, at ocw.mit.edu .

4
00:00:17,000 --> 00:12:59,000
PROFESSOR: Good morning.




In [99]:
course_url = "https://ocw.mit.edu/courses/6-00-introduction-to-computer-science-and-programming-fall-2008/resources/lecture-videos/"
course_name = re.search(r'courses/(.*?)/', course_url).group(1)

with open(f"./{course_name}/{'transcripts'}/lecture-1.txt", 'r') as file:
    srt_content = convert_to_srt(file.read())

with open(f"./{course_name}/{'transcripts'}/lecture-1.srt", 'w') as file:
    file.write(srt_content)

[(1, '0:00', 'The following content is provided under a Creative Commons license.', '0:03'), (2, '0:03', 'Your support will help MIT OpenCourseware continue to offer high-quality educational resources for free.', '0:10'), (3, '0:10', 'To make a donation, or view additional materials from hundreds of MIT courses, visit MIT OpenCourseware, at ocw.mit.edu .', '0:17'), (4, '0:17', 'PROFESSOR: Good morning.', '0:18'), (4, '0:18', 'Try it again.', '0:19'), (4, '0:19', 'Good morning.', '0:22'), (4, '0:22', 'STUDENTS: Good morning.', '0:25'), (4, '0:25', 'PROFESSOR: Thank you.', '0:27'), (4, '0:27', 'This is 6.00, also known as Introduction to Computer', '0:31'), (4, '0:31', 'Science and Programming.', '0:32'), (4, '0:32', "My name is Eric Grimson, I have together Professor John Guttag over here, we're going to be lecturing the course this term.", '0:39'), (5, '0:39', "I want to give you a heads up; you're getting some serious firepower this term.", '0:43'), (6, '0:43', "John was department he